# 1.Loading Data


In [3]:
import pandas as pd

# Load CSV file
robo = pd.read_csv('/content/sample_data/RobotData.csv', header=None)

print(robo.head())
print(robo.info())





          0          1         2   3         4          5        6         7   \
0   3.955138  11.870423 -2.708559   0  1.566351  10.681844  2.69308  6.687333   
1   4.000958   4.238834  1.812858   0  1.566351  10.681844  2.69308  6.687333   
2   8.755436   2.019115 -1.135413   0  1.566351  10.681844  2.69308  6.687333   
3  11.098697   0.086990  0.044513   0  1.566351  10.681844  2.69308  6.687333   
4  10.049046   5.131111 -1.354733   0  1.566351  10.681844  2.69308  6.687333   

         8         9       10        11        12        13  
0  0.187108  1.678316  8.4906  9.150135 -0.118001 -0.054551  
1  0.187108  1.678316  8.4906  9.150135 -0.031162  0.126210  
2  0.187108  1.678316  8.4906  9.150135  0.054829 -0.117872  
3  0.187108  1.678316  8.4906  9.150135  0.129871  0.005785  
4  0.187108  1.678316  8.4906  9.150135  0.027870 -0.126977  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19998 entries, 0 to 19997
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dty

## **The data does not contain any null value**
## **The data contains only continous data, hence no onehot-encoder**

# Preprocessing

Reference: Chat GPT was used for assistance in development of angle_from_robot_to_candle feature during feature engineering

In [8]:
# Preprocessing

import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split


def labeling_data(data):

  #assigning column name to the data
  data.columns = ['X_robot', 'Y_robot', 'Orientation_robot', 'Collision','X_candle1', 'Y_candle1', 'X_candle2', 'Y_candle2','X_candle3', 'Y_candle3',
                'X_candle4', 'Y_candle4','X_speed', 'Y_speed']

def feature_engineering(x):
    x_copy = x.copy()

    # Calculate distance to each candle
    x_copy['distance_from_robot_to_candle1'] = np.sqrt((x_copy['X_robot'] - x_copy['X_candle1'])**2 + (x_copy['Y_robot'] - x_copy['Y_candle1'])**2)
    x_copy['distance_from_robot_to_candle2'] = np.sqrt((x_copy['X_robot'] - x_copy['X_candle2'])**2 + (x_copy['Y_robot'] - x_copy['Y_candle2'])**2)
    x_copy['distance_from_robot_to_candle3'] = np.sqrt((x_copy['X_robot'] - x_copy['X_candle3'])**2 + (x_copy['Y_robot'] - x_copy['Y_candle3'])**2)
    x_copy['distance_from_robot_to_candle4'] = np.sqrt((x_copy['X_robot'] - x_copy['X_candle4'])**2 + (x_copy['Y_robot'] - x_copy['Y_candle4'])**2)

    # Calculate angles to each candle
    x_copy['angle_from_robot_to_candle1'] = np.arctan2(x_copy['Y_candle1'] - x_copy['Y_robot'], x_copy['X_candle1'] - x_copy['X_robot'])
    x_copy['angle_from_robot_to_candle2'] = np.arctan2(x_copy['Y_candle2'] - x_copy['Y_robot'], x_copy['X_candle2'] - x_copy['X_robot'])
    x_copy['angle_from_robot_to_candle3'] = np.arctan2(x_copy['Y_candle3'] - x_copy['Y_robot'], x_copy['X_candle3'] - x_copy['X_robot'])
    x_copy['angle_from_robot_to_candle4'] = np.arctan2(x_copy['Y_candle4'] - x_copy['Y_robot'], x_copy['X_candle4'] - x_copy['X_robot'])




    return x_copy


def augment_data(X, y, noise_factor=0.05):

    # Make a copy of the data to avoid modifying the original
    X_noisy = X.copy()

    # Add noise to X_robot column
    noise_x = np.random.normal(0, noise_factor * X[:, 0].std(), size=len(X[:, 0]))
    X_noisy[:, 0] += noise_x

    # Add noise to Y_Robot column
    noise_y = np.random.normal(0, noise_factor * X[:, 1].std(), size=len(X[:, 1]))
    X_noisy[:, 1] += noise_y

    # Use both original and augmented data points for training
    X_augmented = np.vstack([X_noisy, X])
    y_augmented = np.concatenate([y, y])

    return X_augmented, y_augmented


def replace_missing_values_with_neighbors_data(data, n_neighbors=1):
    df_replaced = data.replace('?', np.nan)
    imputer = KNNImputer(n_neighbors=n_neighbors)
    df_numeric = df_replaced.apply(pd.to_numeric, errors='coerce')
    df_imputed = pd.DataFrame(imputer.fit_transform(df_numeric), columns=df_numeric.columns)
    return df_imputed















# DevelopAndEvaluate

In [15]:


import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split


# if mode is 'develop', it means that we will use the available data for training our model
#if mode is 'evaluate', it means that we will load the model that was trained before (as well as any operators for data pre-processing) to make prediction on novel data
mode = 'develop'

#How many outputs (i.e., columns) your model should predict
num_outputs  = 2

if mode == 'develop':
  #read data from file


  #labeling columns name
  labeling_data(robo)

  #split data into inputs and outputs
  X = robo.drop(['X_speed', 'Y_speed'], axis = 1) # inputs
  y = robo[['X_speed', 'Y_speed']] # outputs

  #feature engineering
  X_feature_engineered = feature_engineering(X)



  # split into train & test sets
  x_train, x_test, y_train, y_test = train_test_split(X_feature_engineered, y, test_size=0.2, random_state=33)

  # Standardise the traing data (similar to what we did in Lecture 4 - pages 15-16)
  scaler = StandardScaler()
  x_train_scaled = scaler.fit_transform(x_train)



  #Train the model
  # we specify that 10% of the training data will be used for validation. Ealy stopping is applied (similar to what we did in Lecture 4, page 33)
  # we also use alpha =0.1 to activate L2 regularisation
  model = MLPRegressor(hidden_layer_sizes=(150,150,150,150,150),learning_rate_init=0.001, batch_size= 1000, activation= 'relu', early_stopping = True, validation_fraction =0.2,alpha =0.1 , verbose = True)
  model.fit(x_train_scaled, y_train)
  train_score = model.score(x_train_scaled, y_train)
  print("Training score:", train_score)
  print("Validation score", model.best_validation_score_)

  # test model
  x_test_scaled = scaler.transform(x_test)

  test_score = model.score(x_test_scaled, y_test)
  print("Testing score:", test_score)





Iteration 1, loss = 0.03968854
Validation score: -0.186676
Iteration 2, loss = 0.03418977
Validation score: 0.072560
Iteration 3, loss = 0.03194501
Validation score: 0.130583
Iteration 4, loss = 0.03001360
Validation score: 0.186572
Iteration 5, loss = 0.02831074
Validation score: 0.236676
Iteration 6, loss = 0.02677198
Validation score: 0.284129
Iteration 7, loss = 0.02539884
Validation score: 0.328768
Iteration 8, loss = 0.02414074
Validation score: 0.369068
Iteration 9, loss = 0.02300856
Validation score: 0.419377
Iteration 10, loss = 0.02197643
Validation score: 0.459296
Iteration 11, loss = 0.02102685
Validation score: 0.503378
Iteration 12, loss = 0.02017943
Validation score: 0.540521
Iteration 13, loss = 0.01938068
Validation score: 0.572976
Iteration 14, loss = 0.01865062
Validation score: 0.597867
Iteration 15, loss = 0.01799245
Validation score: 0.629625
Iteration 16, loss = 0.01735885
Validation score: 0.649165
Iteration 17, loss = 0.01679091
Validation score: 0.658691
Itera

In [16]:
# now save the model, the PCA object, the scaler object & the encoders so that you can load them at a later point & use them for novel data
with open('Final.pkl', 'wb') as f:
  pickle.dump(model, f)



with open('scaler.pkl', 'wb') as f:
  pickle.dump(scaler, f)



# Evaluation

In [ ]:
# Evaluate the model
elif mode == 'evaluate':
  # read the pre-trained model, the PCA object & the scaler object & encoders
  model = pickle.load(open('Final.pkl', 'rb'))
  scaler = pickle.load(open('scaler.pkl', 'rb'))


  # Read novel data. This data usually has no output y. You'll need to use your model to predict y
  novel_data= pd.read_csv('adult2.csv')

  labeling_data(novel_data)

  #split data into inputs and outputs
  X = novel_data.drop(['X_speed', 'Y_speed'], axis = 1) # inputs
  y = novel_data[['X_speed', 'Y_speed']] # outputs

  #feature engineering
  X_feature_engineered = feature_engineering(X)


  X_scaler = scaler.transform(X_feature_engineered)
  # standardise


  model_score = model.score(X_scaler, y)
  print("Model_score: ", model_score)

